# 1 Carga de librerías

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats as stats
from scipy.stats import ttest_ind
from scipy.stats import levene